<a href="https://colab.research.google.com/github/RodrigoGuedesDP/Modelo_Pricing/blob/main/Modelo__LightGBM_Inmobiliaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import joblib
import lightgbm as lgb # Necesita tener lightgbm instalado (pip install lightgbm)

print("--- CARGANDO ARTEFACTOS DEL MODELO ---")
try:

    model = joblib.load('modelo_velocidad_venta.joblib')
    model_columns = joblib.load('model_columns.joblib')
    categorical_info = joblib.load('categorical_info.joblib')
    print("✅ Modelo y artefactos cargados exitosamente.")
except FileNotFoundError:
    print("⚠️ Error: Asegúrate de que los 3 archivos .joblib estén en la misma carpeta que este script.")
    exit()


def predecir_velocidad_venta(datos_departamento):
    """
    Predice la velocidad de venta para un nuevo departamento.
    'datos_departamento' debe ser un diccionario de Python.
    """
    # Crear un DataFrame de una sola fila a partir del diccionario
    df_pred = pd.DataFrame([datos_departamento])

    # Asegurar que el DataFrame tenga todas las columnas que el modelo espera, en el orden correcto
    # y con valores nulos para las que no se especificaron.
    df_pred = df_pred.reindex(columns=model_columns)

    # Convertir las columnas categóricas al tipo 'category' con las categorías correctas
    for col, categories in categorical_info.items():
        # Usamos pd.Categorical para asegurar que el tipo de dato sea el correcto
        df_pred[col] = pd.Categorical(df_pred[col], categories=categories)

    # Realizar la predicción
    prediccion = model.predict(df_pred)
    return prediccion[0]


# --- EJEMPLO DE USO ---
if __name__ == "__main__":
    print("\n--- EJEMPLO DE PREDICCIÓN ---")

    # Tu compañero debe llenar las características del departamento que quiere evaluar.
    # Para las columnas categóricas, debe usar los códigos anónimos.
    # Puede ver las opciones disponibles en el archivo 'categorical_info.joblib' (si decides compartirlo)
    # o puedes darle una lista de las opciones más comunes.
    departamento_a_evaluar = {
        'Inmobiliaria': 'Inmobiliaria_5', # Código anónimo
        'Tipología': 'TIPO A',
        'Distrito': 'Miraflores',
        'Fase_de_Proyecto': 'En Planos',
        'Ratio_Cocheras': 1.5,
        'Cantidad_de_Unidades_Totales': 120,
        'Latitud': -12.12,
        'Longitud': -77.03,
        'Cantidad_Total_de_Estacionamientos': 180,
        'Cantidad_de_Depósitos': 100,
        'Id_Inmobiliaria': 5, # Código anónimo
        'Cantidad_de_Pisos': 15,
        'Precio_de_Lista': 250000
        # No es necesario llenar todas las columnas, las faltantes se tratarán como nulas
    }

    velocidad_estimada = predecir_velocidad_venta(departamento_a_evaluar)

    print(f"\nDatos del departamento a evaluar:")
    for key, value in departamento_a_evaluar.items():
        print(f"  - {key}: {value}")

    print(f"\n🚀 Velocidad de Venta Estimada: {velocidad_estimada:.4f}")

    # También puede ver la importancia de las features (con nombres anonimizados)
    print("\n--- ANÁLISIS: IMPORTANCIA DE CARACTERÍSTICAS ---")
    lgb.plot_importance(model, max_num_features=20, figsize=(10, 8))
    import matplotlib.pyplot as plt
    plt.title("Importancia de Características (Datos Anonimizados)")
    plt.show()